# Uchazeč: data o uchazečích o studium na pedagogických fakultách

- přejmenuj sloupce v datasetu, aby to mělo logičtější strukturu
- připrav variable_labels a value_labels pro export do staty

## Importy

In [1]:
# nejake standardni importy
import os
import sys
import pyreadstat
import pandas as pd
import numpy as np
import re
from statsmodels.stats.weightstats import DescrStatsW
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
# aby grafy byly rozumně čitelné na obrazovce
plt.rcParams['figure.dpi'] = 90
plt.rcParams['axes.titlesize'] = 11
plt.rcParams['figure.figsize'] = 10, 5
#plt.ioff()
import dbf

In [2]:
data_root = '/mnt/d/projects/idea/data'
path17 = 'uchazec/0022MUCH17P'
path21 = 'uchazec/0022MUCH21P'

In [11]:
# načti data z roku 2021
df = pd.read_csv(f'{data_root}/{path21}.csv', encoding='cp1250')

/tmp/ipykernel_461/367580044.py:2: DtypeWarning: Columns (3,6,23,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'{data_root}/{path21}.csv', encoding='cp1250')


In [5]:
df.shape

(284201, 28)

In [2]:
def loader(year=21):
    
    # year can be 17 or 21
    data_root = '/mnt/d/projects/idea/data'
    path = f'uchazec/0022MUCH{year}P'
    
    df = pd.read_csv(f'{data_root}/{path}.csv', encoding='cp1250')
    
    # conversion to numeric
    for c in ['RMAT', 'STAT', 'STATB', 'IZOS', 'VYPR', 'ZAPS'] + [f'APRO{i}' for i in range(1, 4)]:
        df[c] = pd.to_numeric(df[c], errors='coerce')
        
    # strip white spaces
    for c in ['OBSS', 'RID', 'PROGRAM'] + [f'OBOR{i}' for i in range(1, 6)]:
        df[c] = df[c].str.strip()
    
    to_rename = {
        'RDAT': 'dat_sber',
        'RID': 'fak_id',
        'ROD_KOD': 'id',
        'STATB': 'bydliste_stat',
        'OBECB': 'bydliste_obec',
        'PSCB': 'bydliste_psc',
        'STAT': 'stat',
        'ODHL': 'odhl',
        'IZOS': 'ss_izo',
        'OBSS': 'ss_obor',
        'RMAT': 'rmat',
        'TYP_ST': 'typ_st',
        'FORMA_ST': 'forma_st',
        'PROGRAM': 'program',
        'OBOR1': 'obor1',
        'OBOR2': 'obor2',
        'OBOR3': 'obor3',
        'OBOR4': 'obor4',
        'OBOR5': 'obor5',
        'APRO1': 'apro1',
        'APRO2': 'apro2',
        'APRO3': 'apro3',
        'DAT_REG': 'dat_reg',
        'VYPR': 'vypr',
        'DAT_VYPR': 'dat_vypr',
        'ZAPS': 'zaps',
        'DAT_ZAPS': 'dat_zaps'
    }

    to_drop = ['CHYV']
    
    value_labels = {}
    variable_labels = {}
    
    df = df.rename(columns=to_rename).drop(columns=to_drop)
    
    # label STAT, STATB: register AAST
    aast_xml = 'http://stistko.uiv.cz/katalog/textdata/C213436AAST.xml'
    aast = pd.read_xml(aast_xml, encoding='cp1250', xpath='./veta')
    aast['ISO_ALPHA3'] = aast['SPC'].str[2:5]
    df = pd.merge(df, aast[['KOD', 'ISO_ALPHA3']].rename(columns={'KOD': 'bydliste_stat', 'ISO_ALPHA3': 'bydliste_stat_iso'}), 
              how='left')
    df = pd.merge(df, aast[['KOD', 'ISO_ALPHA3']].rename(columns={'KOD': 'stat', 'ISO_ALPHA3': 'stat_iso'}), 
              how='left')
    aast_dict = aast[['KOD', 'ZKR']].set_index('KOD')['ZKR'].to_dict()
    value_labels['stat'] = aast_dict
    value_labels['bydliste_stat'] = aast_dict
    
    registers = {
        'odhl': 'http://stistko.uiv.cz/katalog/textdata/C213729MCPP.xml',
        'typ_st': 'http://stistko.uiv.cz/katalog/textdata/C214019PASP.xml',
        'forma_st' : 'http://stistko.uiv.cz/katalog/textdata/C214048PAFS.xml',
        'vypr': 'http://stistko.uiv.cz/katalog/textdata/C214147MCPR.xml',
        'zaps': 'http://stistko.uiv.cz/katalog/textdata/C21427MCZR.xml',
        # 'OBSS': 'http://stistko.uiv.cz/katalog/textdata/C113922AKSO.xml'
    }
    
    for c, url in registers.items():
        rg = pd.read_xml(url, encoding='cp1250', xpath='./veta')
        rg_dict = rg[['KOD', 'TXT']].set_index('KOD')['TXT'].to_dict()
        value_labels[c] = rg_dict
    
    variable_labels = {
        'id': 'Identifikátor uchazeče (kódované rodné číslo)',
        'stat': 'Státní příslušnost uchazeče',
        'stat_iso': 'Státní příslušnost uchazeče (ISO)',
        'bydliste_stat': 'Stát trvalého pobytu',
        'bydliste_stat_iso': 'Stát trvalého pobytu (ISO)',
        'bydliste_obec': 'Obec trvalého pobytu',
        'bydliste_psc': 'PSČ trvalého pobytu',
        'fak_id': 'Identifikátor fakulty',
        # 'RID_VS_PLN': 'Název vysoké školy',
        # 'RID_ZAR_PLN': 'Název fakulty',
        # 'RID_ZAR_NAZ': 'Zkratka fakulty',
        # 'RID_NUTS': 'Lokalita fakulty',
        'odhl': 'Odkud se uchazeč hlásí',
        'ss_izo': 'Identifikátor střední školy',
        'ss_obor': 'Obor studia střední školy',
        'rmat': 'Rok maturity',
        'typ_st': 'Typ studia',
        'forma_st': 'Forma studia',
        'vypr': 'Výsledek přijímacího řízení',
        'zaps': 'Výsledek zápisu',
        'program': 'Studijní program',
        'obor1': 'Studijní obor',
        'obor2': 'Studijní obor',
        'obor3': 'Studijní obor',
        'obor4': 'Studijní obor',
        'obor5': 'Studijní obor',
        'apro1': 'Aprobace',
        'apro2': 'Aprobace',
        'apro3': 'Aprobace',
        'dat_sber': 'Rozhodné datum sběru',
        'dat_reg': 'Datum podání (registrace) přihlášky',
        'dat_vypr': 'Datum rozhodnutí o výsledku přijímacího řízení',
        'dat_zaps': 'Datum zápisu',
    }

    df = df[variable_labels.keys()]
    
    return df, variable_labels, value_labels
    

In [3]:
df, variable_labels, value_labels = loader()

/tmp/ipykernel_505/3361585551.py:7: DtypeWarning: Columns (3,6,23,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'{data_root}/{path}.csv', encoding='cp1250')


In [6]:
df.to_stata(f'{data_root}/uchazec/uch21.dta', write_index=False, version=118, variable_labels=variable_labels, value_labels=value_labels)

ValueError: Can't create value labels for odhl, value labels can only be applied to numeric columns.

In [42]:
w = pd.io.stata.StataWriterUTF8(f'{data_root}/uchazec/uch21.dta', df, write_index=False, version=118,
                                variable_labels=variable_labels, value_labels=value_labels)
w.write_file()

UnicodeEncodeError: 'latin-1' codec can't encode character '\u0161' in position 11: ordinal not in range(256)

In [43]:
df.to_stata(f'{data_root}/uchazec/uch21.dta', write_index=False, version=118, variable_labels=variable_labels) #, value_labels=value_labels)

In [ ]:
w = pd.io.stata.StataWriterUTF8('foo.dta', df_master)
w.write_file()

In [16]:
df.head()

,RDAT,RID,ROD_KOD,STATB,OBECB,PSCB,STAT,ODHL,IZOS,OBSS,...,OBOR5,APRO1,APRO2,APRO3,DAT_REG,VYPR,DAT_VYPR,ZAPS,DAT_ZAPS,CHYV
0,20211031,1422,,703.0,,91108,703.0,2,999999703.0,,...,,NaN,NaN,NaN,8072021,15,08072021,1,08072021,1 00 0 0 000000 0000000
1,20211031,1423,,703.0,,83107,703.0,2,999999703.0,,...,,NaN,NaN,NaN,23062021,15,23062021,1,23062021,1 00 0 0 000000 0000000
2,20211031,1411,,703.0,,91624,703.0,2,999999703.0,,...,,NaN,NaN,NaN,28072021,15,28072021,1,28072021,1 00 0 0 000000 0000000
3,20211031,1422,,703.0,,91108,703.0,2,999999703.0,,...,,NaN,NaN,NaN,8072021,15,08072021,1,08072021,1 00 0 0 000000 0000000
4,20211031,78900,,566.0,,,566.0,2,999999566.0,,...,,NaN,NaN,NaN,10112020,20,10112020,0,,00 01 1 0000 0000 0 0


In [44]:
pd.__version__

'1.4.3'

In [17]:
aast_xml = 'http://stistko.uiv.cz/katalog/textdata/C213436AAST.xml'
aast = pd.read_xml(aast_xml, encoding='cp1250', xpath='./veta')


In [18]:
aast

,IDC,KOD,ZKR,TXT,DATZP,DATKP,SPC,POZN
0,AAST,0,Zatím nezjištěno,Zatím nezjištěno,1.1.2000,31.12.2999,***************,pouze pro prezentaci
1,AAST,4,Afghánistán,Afghánská islámská republika,1.7.2014,31.12.2999,AFAFG601 5166020,None
2,AAST,8,Albánie,Albánská republika,5.5.2005,31.12.2999,ALALB111 4107015,None
3,AAST,10,Antarktida,Antarktida,5.5.2005,31.12.2999,AQATA961 5089150,None
4,AAST,12,Alžírsko,Alžírská demokratická a lidová republika,1.1.2011,31.12.2999,DZDZA701 5120830,None
...,...,...,...,...,...,...,...,...
247,AAST,876,Wallis a Futuna,Teritorium Wallisovy ostrovy a Futuna,1.7.2014,31.12.2999,WFWLF9933 5081150,None
248,AAST,882,Samoa,Nezávislý stát Samoa,5.5.2005,31.12.2999,WSWSM931 5081950,None
249,AAST,887,Jemen,Jemenská republika,5.5.2005,31.12.2999,YEYEM623 5165320,None
250,AAST,894,Zambie,Zambijská republika,5.5.2005,31.12.2999,ZMZMB785 5137830,None


In [21]:
aast[['KOD', 'ZKR']].set_index('KOD')['ZKR'].to_dict()

{0: 'Zatím nezjištěno',
 4: 'Afghánistán',
 8: 'Albánie',
 10: 'Antarktida',
 12: 'Alžírsko',
 16: 'Americká Samoa',
 20: 'Andorra',
 24: 'Angola',
 28: 'Antigua a Barbuda',
 31: 'Ázerbájdžán',
 32: 'Argentina',
 36: 'Austrálie',
 40: 'Rakousko',
 44: 'Bahamy',
 48: 'Bahrajn',
 50: 'Bangladéš',
 51: 'Arménie',
 52: 'Barbados',
 56: 'Belgie',
 60: 'Bermudy',
 64: 'Bhútán',
 68: 'Bolívie',
 70: 'Bosna a Hercegovina',
 72: 'Botswana',
 74: 'Bouvetův ostrov',
 76: 'Brazílie',
 84: 'Belize',
 86: 'Britské indickooc.území',
 90: 'Šalomounovy ostrovy',
 92: 'Britské Panenské ostr.',
 95: 'Kosovo',
 96: 'Brunej',
 100: 'Bulharsko',
 104: 'Myanmar',
 108: 'Burundi',
 112: 'Bělorusko',
 116: 'Kambodža',
 120: 'Kamerun',
 124: 'Kanada',
 132: 'Kapverdy',
 136: 'Kajmanské ostrovy',
 140: 'Středoafrická republika',
 144: 'Šrí Lanka',
 148: 'Čad',
 152: 'Chile',
 156: 'Čína',
 158: 'Tchaj-wan',
 162: 'Vánoční ostrov',
 166: 'Kokosové (Keeling.) ostr.',
 170: 'Kolumbie',
 174: 'Komory',
 175: 'Mayott

In [7]:
to_rename = {
    'RDAT': 'dat_sber',
    'RID': 'fak_id',
    'ROD_KOD': 'id',
    'STATB': 'bydliste_stat',
    'OBECB': 'bydliste_obec',
    'PSCB': 'bydliste_psc',
    'STAT': 'stat',
    'ODHL': 'odhl',
    'IZOS': 'ss_izo',
    'OBSS': 'ss_obor',
    'RMAT': 'rmat',
    'TYP_ST': 'typ_st',
    'FORMA_ST': 'forma_st',
    'PROGRAM': 'program',
    'OBOR1': 'obor1',
    'OBOR2': 'obor2',
    'OBOR3': 'obor3',
    'OBOR4': 'obor4',
    'OBOR5': 'obor5',
    'APRO1': 'apro1',
    'APRO2': 'apro2',
    'APRO3': 'apro3',
    'DAT_REG': 'dat_reg',
    'VYPR': 'vypr',
    'DAT_VYPR': 'dat_vypr',
    'ZAPS': 'zaps',
    'DAT_ZAPS': 'dat_zaps'
}

to_drop = ['CHYV']

In [12]:
# conversion to numeric
for c in ['RMAT', 'STAT', 'STATB', 'IZOS'] + [f'APRO{i}' for i in range(1, 4)]:
    df[c] = pd.to_numeric(df[c], errors='coerce')

# strip white spaces
for c in ['OBSS', 'RID', 'PROGRAM'] + [f'OBOR{i}' for i in range(1, 6)]:
    df[c] = df[c].str.strip()

df = df.rename(columns=to_rename).drop(columns=to_drop)

In [13]:
df.head()

,dat_sber,fak_id,id,bydliste_stat,bydliste_obec,bydliste_psc,stat,odhl,ss_izo,ss_obor,...,obor4,obor5,apro1,apro2,apro3,dat_reg,vypr,dat_vypr,zaps,dat_zaps
0,20211031,1422,,703.0,,91108,703.0,2,999999703.0,,...,,,NaN,NaN,NaN,8072021,15,08072021,1,08072021
1,20211031,1423,,703.0,,83107,703.0,2,999999703.0,,...,,,NaN,NaN,NaN,23062021,15,23062021,1,23062021
2,20211031,1411,,703.0,,91624,703.0,2,999999703.0,,...,,,NaN,NaN,NaN,28072021,15,28072021,1,28072021
3,20211031,1422,,703.0,,91108,703.0,2,999999703.0,,...,,,NaN,NaN,NaN,8072021,15,08072021,1,08072021
4,20211031,78900,,566.0,,,566.0,2,999999566.0,,...,,,NaN,NaN,NaN,10112020,20,10112020,0,


## Loader

In [3]:
def loader(year=17):
    # year can be 17 or 21
    data_root = '/mnt/d/projects/idea/data'
    path = f'uchazec/0022MUCH{year}P'
    
    df = pd.read_csv(f'{data_root}/{path}.csv', encoding='cp1250')

    # conversion to numeric
    for c in ['RMAT', 'STAT', 'STATB', 'IZOS'] + [f'APRO{i}' for i in range(1, 4)]:
        df[c] = pd.to_numeric(df[c], errors='coerce')
        
    # strip white spaces
    for c in ['OBSS', 'RID', 'PROGRAM'] + [f'OBOR{i}' for i in range(1, 6)]:
        df[c] = df[c].str.strip()
    
    # label STAT, STATB: register AAST
    aast_xml = 'http://stistko.uiv.cz/katalog/textdata/C213436AAST.xml'
    aast = pd.read_xml(aast_xml, encoding='cp1250', xpath='./veta')
    aast['ISO_ALPHA3'] = aast['SPC'].str[2:5]
    df = pd.merge(df, aast[['KOD', 'ZKR', 'ISO_ALPHA3']].rename(columns={'KOD': 'STAT', 'ZKR': 'STAT_LABEL', 'ISO_ALPHA3': 'STAT_ISO'}), 
              how='left')
    df = pd.merge(df, aast[['KOD', 'ZKR', 'ISO_ALPHA3']].rename(columns={'KOD': 'STATB', 'ZKR': 'STATB_LABEL', 'ISO_ALPHA3': 'STATB_ISO'}), 
              how='left')
    
    # label OBSS - I am not including full school info
    #   -> actually, this give me slightly better match - probably because I am including also specializations that are not valid anymore
    li = []
    for y in range(1999, 2023):
        pdf = pd.read_csv(f'{data_root}/uchazec/stredni-skoly/m{y}.csv', encoding='cp1250')
        li.append(pdf)
    ss = pd.concat(li, axis=0, ignore_index=True).drop_duplicates().reset_index(drop=True)
    
    def selector(frame):
        # just use the longest one - the shorter ones typically use abbreviations
        frame['len'] = frame['PLN_NAZ'].str.strip().str.len()
        return frame.sort_values('len', ascending=False)['PLN_NAZ'].iloc[0]

    ss['OBOR'] = ss['OBOR'].str.strip()
    obory = ss[['OBOR', 'PLN_NAZ']].drop_duplicates()
    obory_uni = obory.groupby('OBOR')[['PLN_NAZ']].apply(selector).reset_index()
    df = pd.merge(df, obory_uni.rename(columns={'OBOR': 'OBSS', 0: 'OBSS_LABEL'}), how='left')
    
    sss = ss[['IZO', 'ZAR_PLN', 'VUSC']].drop_duplicates()
    df = pd.merge(df, sss.rename(columns={'IZO': 'IZOS', 'ZAR_PLN': 'IZOS_LABEL', 'VUSC': 'IZOS_NUTS'}), how='left')
    
    registers = {
        'ODHL': 'http://stistko.uiv.cz/katalog/textdata/C213729MCPP.xml',
        'TYP_ST': 'http://stistko.uiv.cz/katalog/textdata/C214019PASP.xml',
        'FORMA_ST' : 'http://stistko.uiv.cz/katalog/textdata/C214048PAFS.xml',
        'VYPR': 'http://stistko.uiv.cz/katalog/textdata/C214147MCPR.xml',
        'ZAPS': 'http://stistko.uiv.cz/katalog/textdata/C21427MCZR.xml',
        # 'OBSS': 'http://stistko.uiv.cz/katalog/textdata/C113922AKSO.xml'
    }
    
    for c, url in registers.items():
        rg = pd.read_xml(url, encoding='cp1250', xpath='./veta')
        df = pd.merge(df, rg[['KOD', 'TXT']].rename(columns={'KOD': c, 'TXT': f'{c}_LABEL'}), how='left')

    regpro = pd.read_csv(f'{data_root}/uchazec/uch3-03-6/cisel/regpro.csv', encoding='cp1250')
    regpro['RID'] = regpro['RID'].str.strip()
    regpro_cols = ['ZAR_PLN', 'VS_PLN', 'ZAR_NAZ', 'UZEMI']
    regpro = regpro[['RID'] + regpro_cols].rename(columns={x: f'RID_{x}' for x in regpro_cols})
    df = pd.merge(df, regpro.rename(columns={'RID_UZEMI': 'RID_NUTS'}), how='left')
    
    program = pd.read_csv(f'{data_root}/uchazec/uch3-03-6/cisel/program.csv', encoding='cp1250')
    df = pd.merge(df, program.rename(columns={'KOD': 'PROGRAM', 'NAZEV': 'PROGRAM_LABEL'}), how='left')
    
    # plné obory
    akvo_xml = 'http://stistko.uiv.cz/katalog/textdata/C214117AKVO.xml'
    akvo = pd.read_xml(akvo_xml, encoding='cp1250', xpath='./veta')
    for i in range(1, 6):
        df = pd.merge(df, akvo[['KOD', 'TXT']].rename(columns={'KOD': f'OBOR{i}', 'TXT': f'OBOR{i}_LABEL'}), how='left')
        
    # aprobace
    aaap_xml = 'http://stistko.uiv.cz/katalog/textdata/C214234AAAP.xml'
    aaap = pd.read_xml(aaap_xml, encoding='cp1250', xpath='./veta')
    for i in range(1, 4):
        df = pd.merge(df, aaap[['KOD', 'TXT']].rename(columns={'KOD': f'APRO{i}', 'TXT': f'APRO{i}_LABEL'}), how='left')
        
    # studijní obory podle skupin
    akko_xml = 'http://stistko.uiv.cz/katalog/textdata/C11240AKKO.xml'
    akko = pd.read_xml(akko_xml, encoding='cp1250', xpath='./veta')
    for i in [1, 2, 5]:
        df[f'OBOR1_{i}'] = df['OBOR1'].str[:i]
        df = pd.merge(df, akko[['KOD', 'TXT']].rename(columns={'KOD': f'OBOR1_{i}', 'TXT': f'OBOR1_{i}_LABEL'}), how='left')
    
    df['TYP_SS'] = np.where(df['OBSS'] == '', 'JINE', np.where(df['OBSS'].str.startswith('794'), 'G', 'SOS'))
    df['TYP_G'] = np.where(df['TYP_SS'] != 'G', np.nan, df['OBSS'].str[5])
    
    # fillna in some columns
    for c in ['PROGRAM_LABEL'] + [f'OBOR{i}_LABEL' for i in range(1, 6)] + [f'APRO{i}_LABEL' for i in range(1, 4)]:
        df[c] = df[c].fillna('')
    
    return df

In [4]:
df = loader()

/tmp/ipykernel_384/678624935.py:6: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'{data_root}/{path}.csv', encoding='cp1250')


In [5]:
df.head()

,RDAT,RID,ROD_KOD,STATB,OBECB,PSCB,STAT,ODHL,IZOS,OBSS,...,APRO2_LABEL,APRO3_LABEL,OBOR1_1,OBOR1_1_LABEL,OBOR1_2,OBOR1_2_LABEL,OBOR1_5,OBOR1_5_LABEL,TYP_SS,TYP_G
0,20171031,78900,,840.0,,,840.0,2,999999840.0,,...,,,6,"SPOLEČENSKÉ VĚDY, NAUKY A SLUŽBY (1. část)",67,Sociální vědy,6701R,Politologie,JINE,NaN
1,20171031,78900,,368.0,,,368.0,2,999999368.0,,...,,,8,VĚDY A NAUKY O KULTUŘE A UMĚNÍ,82,Umění a užité umění,8206R,Výtvarná umění,JINE,NaN
2,20171031,15110,000101,112.0,,,112.0,2,999999112.0,,...,,,5,"ZDRAVOTNICTVÍ, LÉKAŘSKÉ A FARMACEUTICKÉ VĚDY A...",51,Lékařské vědy,5103T,Všeobecné lékařství,JINE,NaN
3,20171031,15110,000101,112.0,,,112.0,2,999999112.0,,...,,,5,"ZDRAVOTNICTVÍ, LÉKAŘSKÉ A FARMACEUTICKÉ VĚDY A...",51,Lékařské vědy,5104T,Stomatologie,JINE,NaN
4,20171031,14110,000101,112.0,,23104,112.0,9,999999112.0,,...,,,5,"ZDRAVOTNICTVÍ, LÉKAŘSKÉ A FARMACEUTICKÉ VĚDY A...",51,Lékařské vědy,5103T,Všeobecné lékařství,JINE,NaN


In [6]:
akko_xml = 'http://stistko.uiv.cz/katalog/textdata/C11240AKKO.xml'
akko = pd.read_xml(akko_xml, encoding='cp1250', xpath='./veta')
akko.head()

,IDC,KOD,ZKR,TXT,DATZP,DATKP,SPC,POZN
0,AKKO,1,PŘÍRODNÍ VĚDY A NAUKY,PŘÍRODNÍ VĚDY A NAUKY,1.9.1998,31.12.2999,NaN,None
1,AKKO,11,Matematické obory,Matematické obory,1.9.1998,31.12.2999,NaN,None
2,AKKO,1101R,Matematika,Matematika,1.9.1998,31.12.2999,NaN,None
3,AKKO,1101T,Matematika,Matematika,1.9.1998,31.12.2999,NaN,None
4,AKKO,1101V,Matematika,Matematika,1.9.1998,31.12.2999,NaN,None


In [8]:
akko['KOD'].str.len()

0       1
1       2
2       5
3       5
4       5
       ..
1155    5
1156    2
1157    5
1158    5
1159    5
Name: KOD, Length: 1160, dtype: int64